<a href="https://colab.research.google.com/github/Jawakar-7/Image-Classification-using-multiple-models-/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Jawakar-7/Image-Classification-using-multiple-models-.git

Cloning into 'Image-Classification-using-multiple-models-'...
remote: Enumerating objects: 24370, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 24370 (delta 19), reused 21 (delta 7), pack-reused 24316
Receiving objects: 100% (24370/24370), 369.25 MiB | 16.82 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Updating files: 100% (24344/24344), done.


In [2]:


import seaborn as sns
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import confusion_matrix,classification_report

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
test_path='/content/Image-Classification-using-multiple-models-/Datasets/seg_test/seg_test'
train_path='/content/Image-Classification-using-multiple-models-/Datasets/seg_train/seg_train'


In [4]:
from sklearn.utils import column_or_1d
train_xgen=ImageDataGenerator(
    rescale=1.0/255,#Scaling Down the data
    zoom_range=0.1,
    width_shift_range=0.2,
    height_shift_range=0.1,
)
X_train=train_xgen.flow_from_directory(
    train_path,
    target_size=(150,150),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32
)

Found 14034 images belonging to 6 classes.


In [5]:
test_xgen=ImageDataGenerator(
    rescale=1.0/255
)
X_test=test_xgen.flow_from_directory(
    test_path,
    class_mode='categorical',
    target_size=(150,150),
    batch_size=32,
    color_mode='rgb',
    shuffle=True
)

Found 3000 images belonging to 6 classes.


In [6]:

labels={i:k for k,i in X_train.class_indices.items()}
for i,k in labels.items():
  print(f"{i}:{k}")


0:buildings
1:forest
2:glacier
3:mountain
4:sea
5:street


In [7]:
print(f"{len(X_train)} which is total images/32")

439 which is total images/32


In [8]:

for i,k in X_train:
  print(f"{i.shape}is a tuple that contains(batch size,target size and channel_size,) and k is the corresponding classe of  image {labels[np.argmax(k[0],axis=0)]} meaning the last class")
  break


(32, 150, 150, 3)is a tuple that contains(batch size,target size and channel_size,) and k is the corresponding classe of  image mountain meaning the last class


In [9]:
X_train[0][1][0]#accesing the label of first element from the first batch

array([0., 0., 0., 1., 0., 0.], dtype=float32)

## Inception V3

In [18]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

# local_weights_file = '/kaggle/input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                include_top = False,
                                weights = "imagenet")

# pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
     layer.trainable = False

# pre_trained_model.summary()
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(6, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001),
              loss = CategoricalCrossentropy(),
              metrics = ['acc'])

history=model.fit(X_train,epochs=5,validation_data=X_test)

last layer output shape:  (None, 7, 7, 768)
Epoch 1/5
439/439 [==============================] - 97s 214ms/step - loss: 0.4370 - acc: 0.8468 - val_loss: 0.2609 - val_acc: 0.9017
Epoch 2/5
439/439 [==============================] - 91s 208ms/step - loss: 0.3075 - acc: 0.8897 - val_loss: 0.2400 - val_acc: 0.9170
Epoch 3/5
439/439 [==============================] - 93s 212ms/step - loss: 0.2657 - acc: 0.9033 - val_loss: 0.2323 - val_acc: 0.9123
Epoch 4/5
439/439 [==============================] - 91s 208ms/step - loss: 0.2403 - acc: 0.9141 - val_loss: 0.2368 - val_acc: 0.9170
Epoch 5/5
439/439 [==============================] - 92s 209ms/step - loss: 0.2270 - acc: 0.9200 - val_loss: 0.2432 - val_acc: 0.9253


# VGG 16

In [19]:
from tensorflow.keras.applications import VGG16

pretrained_model=VGG16(input_shape = (150, 150, 3),
                        include_top = False,
                        weights = 'imagenet')

for layer in pretrained_model.layers:
     layer.trainable = False

# pretrained_model.summary()
last_layer = pretrained_model.get_layer('block5_pool')
print('last layer of vgg : output shape: ', last_layer.output_shape)
last_output= last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(6, activation='softmax')(x)

model_vgg = Model(pretrained_model.input, x)


model_vgg.compile(optimizer = RMSprop(learning_rate=0.0001),
              loss = CategoricalCrossentropy(),
              metrics = ['acc'])

model_vgg.fit(X_train,epochs=5,validation_data=X_test)

58889256/58889256 [==============================] - 3s 0us/step
last layer of vgg : output shape:  (None, 4, 4, 512)
Epoch 1/5
439/439 [==============================] - 107s 235ms/step - loss: 0.5808 - acc: 0.7805 - val_loss: 0.3633 - val_acc: 0.8650
Epoch 2/5
439/439 [==============================] - 101s 230ms/step - loss: 0.4254 - acc: 0.8447 - val_loss: 0.3860 - val_acc: 0.8580
Epoch 3/5
439/439 [==============================] - 99s 225ms/step - loss: 0.3932 - acc: 0.8554 - val_loss: 0.3782 - val_acc: 0.8580
Epoch 4/5
439/439 [==============================] - 100s 227ms/step - loss: 0.3627 - acc: 0.8677 - val_loss: 0.3234 - val_acc: 0.8787
Epoch 5/5
439/439 [==============================] - 103s 234ms/step - loss: 0.3499 - acc: 0.8709 - val_loss: 0.3318 - val_acc: 0.8787


## Resnet 50

In [20]:
from tensorflow.keras.applications import ResNet50

#step1
# file_resnet='/kaggle/input/vgg16/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
pretrained_model=ResNet50( input_shape=(150,150,3),
                                  include_top=False,
                                  weights='imagenet'
                                   )
#step2
for layer in pretrained_model.layers:
     layer.trainable = False

# pretrained_model.summary()

#step3
last_layer = pretrained_model.get_layer('conv5_block3_out')
print('last layer of vgg : output shape: ', last_layer.output_shape)
last_output = last_layer.output

#step4
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(6, activation='softmax')(x)

#step5
model_resnet = Model(pretrained_model.input, x)

#step6
model_resnet.compile(optimizer = RMSprop(learning_rate=0.0001),
              loss = CategoricalCrossentropy(),
              metrics = ['acc'])
model_resnet.fit(X_train,epochs=5,validation_data=X_test)

94765736/94765736 [==============================] - 5s 0us/step
last layer of vgg : output shape:  (None, 5, 5, 2048)
Epoch 1/5
439/439 [==============================] - 105s 231ms/step - loss: 1.5661 - acc: 0.3670 - val_loss: 1.3604 - val_acc: 0.3933
Epoch 2/5
439/439 [==============================] - 101s 231ms/step - loss: 1.2316 - acc: 0.4857 - val_loss: 1.1279 - val_acc: 0.5270
Epoch 3/5
439/439 [==============================] - 101s 229ms/step - loss: 1.1449 - acc: 0.5289 - val_loss: 1.0718 - val_acc: 0.5647
Epoch 4/5
439/439 [==============================] - 101s 230ms/step - loss: 1.0859 - acc: 0.5594 - val_loss: 0.9812 - val_acc: 0.6013
Epoch 5/5
439/439 [==============================] - 102s 233ms/step - loss: 1.0524 - acc: 0.5764 - val_loss: 1.0425 - val_acc: 0.5663
